In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))

In [ ]:
train = pd.read_csv('../input/train.csv', nrows = 10_000_000)
test = pd.read_csv('../input/test.csv')

In [ ]:
train.pop('key')
train.pop('pickup_datetime')
import gc
gc.collect()

In [ ]:
kID = test.pop('key')
test.pop('pickup_datetime')
import gc
gc.collect()

In [ ]:
train = train[train['fare_amount']>0]
train = train[train['fare_amount']<300]

In [ ]:
train.shape

In [ ]:
train.isnull().values.any()

In [ ]:
train = train.dropna()
train.shape

In [ ]:
test.isnull().values.any()

In [ ]:
print(train.isnull().sum())

In [ ]:
y = train.pop('fare_amount')

In [ ]:
val = train.shape[0]
test.shape, train.shape

In [ ]:
train = pd.concat([train,test])
pas = train['passenger_count']
train.shape

In [ ]:
R = 6373.0

lat1 = train['pickup_latitude']
lon1 = train['pickup_longitude']
lat2 = train['dropoff_latitude']
lon2 = train['dropoff_longitude']
dlon = lon2 - lon1
dlat = lat2 - lat1
a = ((np.sin(dlat/2.0))**2) + (np.cos(lat1) * np.cos(lat2) * (np.sin(dlon/2.0))**2)
c = 2.0 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
distance = R * c
del R,c,a,dlon,dlat
distance.shape

In [ ]:
#train = pd.get_dummies(data=train, columns=['passenger_count'])
train.shape

In [ ]:
del train
train = pd.DataFrame(data=np.column_stack([distance,pas]))
train.head()

In [ ]:
test = train[val:train.shape[0]]
train = train[0:val]
train.shape, test.shape

In [ ]:
from sklearn.metrics import mean_squared_error as msq

In [ ]:
import h2o
h2o.init(max_mem_size = 9)
h2o.remove_all()

In [ ]:
train['fare_amount'] = y

In [ ]:
tr = h2o.H2OFrame(train.values)

In [ ]:
del train
gc.collect()

In [ ]:
ts = h2o.H2OFrame(test.values)

In [ ]:
del test
gc.collect()

In [ ]:
tr.head()

In [ ]:
from h2o.automl import H2OAutoML

In [ ]:
model = H2OAutoML()

In [ ]:
model.train(ts.col_names, 'C3', tr)

In [ ]:
pred = model.predict(ts)

In [ ]:
pred = pred.as_data_frame()

In [ ]:
pred.head()

In [ ]:
submission = pd.DataFrame({
        "key": kID,
        "fare_amount": pred.values
})

submission.to_csv('FARES.csv',index=False)